In [27]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
import torchvision
import glob
import torch.nn.functional as F

In [28]:
classes = ['f','j','k','l','m','n','o','x','y','z']

transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

test_path = 'data_letters/test'

dataset = torchvision.datasets.ImageFolder(test_path, transform=transformer)

dataloader = DataLoader(
    dataset,
    batch_size = 256,
    shuffle = False
)

In [31]:
class Task1Res(nn.Module):
    def __init__(self, in_channel, out_channel, stride=1):
        super(Task1Res, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channel)
        )
        self.shortcut = nn.Sequential()
        if in_channel != out_channel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channel)
            )

    def forward(self, x):
        out = self.conv(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class Task1Net(nn.Module):
    def __init__(self, Task1Res, num_classes=10):
        super(Task1Net, self).__init__()
        self.in_channel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.conv2 = self.layer(Task1Res, 64,  2, stride=1)
        self.fc = nn.Linear(in_features=64*16*16, out_features=num_classes)

    def layer(self, block, channels, num_blocks, stride):
#         strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        layers.append(block(self.in_channel, channels, stride))
        layers.append(block(self.in_channel, channels, stride))
#         for stride in strides:
#             layers.append(block(self.in_channel, channels, stride))
#             self.in_channel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def MyNet():
    return Task1Net(Task1Res)

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyNet()
# model = torchvision.models.resnet34(pretrained=True)
# inchannel = model.fc.in_features
# model.fc = nn.Linear(inchannel, 10)
best_model = torch.load('model/task1-adam-decay.model')
model.load_state_dict(best_model, strict=False)
model.to(device)
count = len(dataset)
model.eval()

sum_accuracy = 0.0
for i, (images, labels) in enumerate(dataloader):
    if torch.cuda.is_available():
        images = images.to(device)
        labels = labels.to(device)
    outputs = model(images)
    _,prediction = torch.max(outputs.data,1)
    sum_accuracy += int(torch.sum(prediction==labels.data))

accuracy = sum_accuracy / count

print(f'Accuracy: ', accuracy)

Accuracy:  0.798733604703754
